In [ ]:
# ====== Imports ======
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import iqr

In [ ]:
# ====== Device ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# ==================== Load Data ====================
file_path = '/content/spectra_train_795.csv'
X_df = pd.read_csv(file_path)
print("df shape:", X_df.shape)

df shape: (795, 257)


In [ ]:
y_df =  pd.read_csv("/content/target_train_795.csv")

print("y shape:", y_df.shape)

y shape: (795, 3)


In [ ]:
concen_columns = ['Moi','NDF','Starch']

In [ ]:
# ====== Scaling ======
def autoscale(X):
    """Applies Autoscale (mean centering and unit variance scaling) to the input data."""
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    # Avoid division by zero for columns with zero standard deviation
    std[std == 0] = 1e-8
    return (X - mean) / std

X_scaled = autoscale(X_df.values)
X = torch.tensor(X_scaled, dtype=torch.float32).unsqueeze(1)  # [N,1,L]

y_scaled = autoscale(y_df.values)
y = torch.tensor(y_scaled, dtype=torch.float32)  # [N, num_targets]

# ====== Check input length ======
input_length = X.shape[2]
print("Input length:", input_length)

Input length: 257


In [ ]:
# ====== Hyperparameters ======
batch_size = 50 #128
epochs = 200
learning_rate = 0.0001 #0.001
n_splits = 5

In [ ]:
class CNNWrapper(nn.Module):
    def __init__(self, mode="1d", input_length=257, num_classes=1):
        super(CNNWrapper, self).__init__()
        self.mode = mode

        if mode == "1d":
            self.conv1 = nn.Conv1d(1, 64, 3, padding=1)
            self.bn1 = nn.BatchNorm1d(64)
            self.pool = nn.MaxPool1d(2)

            self.conv2 = nn.Conv1d(64, 128, 3, padding=1)
            self.bn2 = nn.BatchNorm1d(128)

            self.conv3 = nn.Conv1d(128, 256, 3, padding=1)
            self.bn3 = nn.BatchNorm1d(256)
            self.conv4 = nn.Conv1d(256, 256, 3, padding=1)
            self.bn4 = nn.BatchNorm1d(256)

            self.conv5 = nn.Conv1d(256, 512, 3, padding=1)
            self.bn5 = nn.BatchNorm1d(512)
            self.conv6 = nn.Conv1d(512, 512, 3, padding=1)
            self.bn6 = nn.BatchNorm1d(512)

            self.conv7 = nn.Conv1d(512, 512, 3, padding=1)
            self.bn7 = nn.BatchNorm1d(512)
            self.conv8 = nn.Conv1d(512, 512, 3, padding=1)
            self.bn8 = nn.BatchNorm1d(512)

            flatten_dim = (input_length // (2**5)) * 512
            self.fc = nn.Linear(flatten_dim, num_classes)
        else:
            raise ValueError("Only 1D CNN implemented as in paper")

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))); x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x))); x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x))); x = F.relu(self.bn4(self.conv4(x))); x = self.pool(x)
        x = F.relu(self.bn5(self.conv5(x))); x = F.relu(self.bn6(self.conv6(x))); x = self.pool(x)
        x = F.relu(self.bn7(self.conv7(x))); x = F.relu(self.bn8(self.conv8(x))); x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
# ====== Train/Test Split ======
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
print("Training+CV samples:", X_train_val.shape[0])
print("Test samples:", X_test.shape[0])

Training+CV samples: 596
Test samples: 199


In [ ]:
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []
calibration_results = [] # To store calibration metrics

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
    print(f"\n=== Fold {fold+1} Training ===")

    X_train, X_val = X_train_val[train_idx].to(device), X_train_val[val_idx].to(device)
    y_train, y_val = y_train_val[train_idx].to(device), y_train_val[val_idx].to(device)

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model = CNNWrapper(mode="1d", input_length=input_length, num_classes=y.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # -------- Training --------
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            # Add gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        if (epoch + 1) % 10 == 0:  # Print loss every 10 epochs
            print(f"Fold {fold+1} - Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

    # -------- Validation --------
    model.eval()
    with torch.no_grad():
        y_pred_val = model(X_val).cpu().numpy()
        y_true_val = y_val.cpu().numpy()

    fold_rmse, fold_r2, fold_rpiq = [], [], []
    print(f"\n=== Fold {fold+1} Validation Metrics ===")
    for i, col in enumerate(concen_columns):
        rmse = np.sqrt(mean_squared_error(y_true_val[:, i], y_pred_val[:, i]))
        r2 = r2_score(y_true_val[:, i], y_pred_val[:, i])
        rpiq = iqr(y_true_val[:, i]) / rmse
        fold_rmse.append(rmse)
        fold_r2.append(r2)
        fold_rpiq.append(rpiq)
        print(f"{col}: RMSE={rmse:.4f}, R²={r2:.4f}, RPIQ={rpiq:.4f}")

    fold_results.append((fold_rmse, fold_r2, fold_rpiq))

    # -------- Calibration Metrics (on Training Data from the last epoch) --------
    model.eval()
    with torch.no_grad():
        y_pred_train = model(X_train).cpu().numpy()
        y_true_train = y_train.cpu().numpy()

    cal_rmse, cal_r2, cal_bias = [], [], []
    print(f"\n=== Fold {fold+1} Calibration Metrics (Last Epoch) ===")
    for i, col in enumerate(concen_columns):
        rmse = np.sqrt(mean_squared_error(y_true_train[:, i], y_pred_train[:, i]))
        r2 = r2_score(y_true_train[:, i], y_pred_train[:, i])
        bias = np.mean(y_pred_train[:, i] - y_true_train[:, i]) # Calculate bias
        cal_rmse.append(rmse)
        cal_r2.append(r2)
        cal_bias.append(bias)
        print(f"{col}: RMSE={rmse:.4f}, R²={r2:.4f}, Bias={bias:.4f}")

    calibration_results.append((cal_rmse, cal_r2, cal_bias))


=== Fold 1 Training ===
Fold 1 - Epoch 10/200, Loss: 0.324709
Fold 1 - Epoch 20/200, Loss: 0.187434
Fold 1 - Epoch 30/200, Loss: 0.167187
Fold 1 - Epoch 40/200, Loss: 0.139671
Fold 1 - Epoch 50/200, Loss: 0.106191
Fold 1 - Epoch 60/200, Loss: 0.097723
Fold 1 - Epoch 70/200, Loss: 0.091458
Fold 1 - Epoch 80/200, Loss: 0.055327
Fold 1 - Epoch 90/200, Loss: 0.064704
Fold 1 - Epoch 100/200, Loss: 0.056893
Fold 1 - Epoch 110/200, Loss: 0.032659
Fold 1 - Epoch 120/200, Loss: 0.029059
Fold 1 - Epoch 130/200, Loss: 0.025693
Fold 1 - Epoch 140/200, Loss: 0.025826
Fold 1 - Epoch 150/200, Loss: 0.027079
Fold 1 - Epoch 160/200, Loss: 0.022228
Fold 1 - Epoch 170/200, Loss: 0.024315
Fold 1 - Epoch 180/200, Loss: 0.020518
Fold 1 - Epoch 190/200, Loss: 0.018850
Fold 1 - Epoch 200/200, Loss: 0.018011

=== Fold 1 Validation Metrics ===
Moi: RMSE=0.3181, R²=0.9066, RPIQ=3.2947
NDF: RMSE=0.4836, R²=0.7597, RPIQ=2.3833
Starch: RMSE=0.4288, R²=0.8562, RPIQ=2.4665

=== Fold 1 Calibration Metrics (Last Epoch

In [ ]:
# ====== Average Calibration Metrics ======
avg_rmse_cal = np.mean([r[0] for r in calibration_results], axis=0)
avg_r2_cal = np.mean([r[1] for r in calibration_results], axis=0)
avg_bias_cal = np.mean([r[2] for r in calibration_results], axis=0)

# Create DataFrame for average Calibration metrics
avg_calibration_metrics = []
for i, col in enumerate(concen_columns):
    avg_calibration_metrics.append({'Property': col, 'Avg RMSE': avg_rmse_cal[i], 'Avg R²': avg_r2_cal[i], 'Avg Bias': avg_bias_cal[i]})
avg_calibration_metrics_df = pd.DataFrame(avg_calibration_metrics)
# Reorder columns for calibration metrics
avg_calibration_metrics_df = avg_calibration_metrics_df[['Property', 'Avg R²', 'Avg RMSE', 'Avg Bias']]

print("\n=== Average Calibration Metrics ===")
display(avg_calibration_metrics_df)

# ====== Average CV Metrics ======
avg_rmse_cv = np.mean([r[0] for r in fold_results], axis=0)
avg_r2_cv = np.mean([r[1] for r in fold_results], axis=0)
# avg_rpiq_cv = np.mean([r[2] for r in fold_results], axis=0) # Removed RPIQ

# Calculate average bias for CV
avg_bias_cv = np.mean([r[2] for r in fold_results], axis=0)


# Create DataFrame for average CV metrics
avg_cv_metrics = []
for i, col in enumerate(concen_columns):
    # avg_cv_metrics.append({'Property': col, 'Avg RMSE': avg_rmse_cv[i], 'Avg R²': avg_r2_cv[i], 'Avg RPIQ': avg_rpiq_cv[i]}) # Removed RPIQ
     avg_cv_metrics.append({'Property': col, 'Avg RMSE': avg_rmse_cv[i], 'Avg R²': avg_r2_cv[i], 'Avg Bias': avg_bias_cv[i]}) # Added back without RPIQ
avg_cv_metrics_df = pd.DataFrame(avg_cv_metrics)
# Reorder columns for CV metrics
avg_cv_metrics_df = avg_cv_metrics_df[['Property', 'Avg R²', 'Avg RMSE', 'Avg Bias']]


print("\n=== Average CV Metrics ===")
display(avg_cv_metrics_df)

# ====== Evaluate on Test Set ======
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test.to(device)).cpu().numpy()
    y_test_true = y_test.numpy()

print("\n=== Test Set Metrics ===")
test_metrics = []
for i, col in enumerate(concen_columns):
    rmse = np.sqrt(mean_squared_error(y_test_true[:, i], y_test_pred[:, i]))
    r2 = r2_score(y_test_true[:, i], y_test_pred[:, i])
    bias = np.mean(y_test_pred[:, i] - y_test_true[:, i]) # Calculate bias
    test_metrics.append({'Property': col, 'RMSE': rmse, 'R²': r2, 'Bias': bias})

test_metrics_df = pd.DataFrame(test_metrics)
# Reorder columns for test metrics
test_metrics_df = test_metrics_df[['Property', 'R²', 'RMSE', 'Bias']]
display(test_metrics_df)


=== Average Calibration Metrics ===


,Property,Avg R²,Avg RMSE,Avg Bias
0,Moi,0.986507,0.109864,0.016251
1,NDF,0.977617,0.144221,0.027059
2,Starch,0.981941,0.127122,-0.022800



=== Average CV Metrics ===


,Property,Avg R²,Avg RMSE,Avg Bias
0,Moi,0.881137,0.325698,3.300024
1,NDF,0.610693,0.610640,2.016965
2,Starch,0.773102,0.450834,2.184307



=== Test Set Metrics ===


,Property,R²,RMSE,Bias
0,Moi,0.936391,0.281541,0.020455
1,NDF,0.823591,0.430890,0.160942
2,Starch,0.892821,0.354828,-0.103080


In [ ]:
print(f"Average R² (Calibration): {avg_r2_cal.mean():.4f}")
print(f"Average R² (CV): {avg_r2_cv.mean():.4f}")
print(f"Average R² (Test): {test_metrics_df['R²'].mean():.4f}")

Average R² (Calibration): 0.9820
Average R² (CV): 0.7550
Average R² (Test): 0.8843


In [51]:
# Save model state_dict (weights only)
model_save_path = "1d_cnn_model_weights.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model weights saved to {model_save_path}")

# # Later: Load weights into the same model architecture
# loaded_model = CNNWrapper(mode="1d", input_length=input_length, num_classes=y.shape[1]).to(device)
# loaded_model.load_state_dict(torch.load(model_save_path, map_location=device))
# loaded_model.eval()
# print("Model weights loaded successfully")

Model weights saved to 1d_cnn_model_weights.pth
Model weights loaded successfully
